In [3]:
from DH import *
import socket
import threading
import datetime

class ThreadedServer():
    def __init__(self,public_key, private_key, host = '', port = 9091):
        self.host = host
        self.port = port
        self.public_key = public_key
        self.private_key = private_key
        self.socket = socket.socket()
        self.socket.bind((self.host, self.port))
    
    def client_processing(self, conn, addr, data_batch = 1024):
        while True:
            data = conn.recv(data_batch)
            print(f'{datetime.datetime.now()} INFO: Получены данные от клиента:', data.decode('utf-8'))
            if data.decode('utf-8') == 'get_public_key':
                conn.send(bytes(self.public_key, 'utf-8'))
            if (str(data.decode('utf-8')).split(' '))[0] == 'public_key':
                client_public_key = int(str(data.decode('utf-8'))[1])
                shr = DH(self.public_key, client_public_key, self.private_key)
                partial_key = shr.generate_partial_key()
                conn.send(bytes(partial_key, 'utf-8'))
            if (str(data.decode('utd-8')).split(' '))[0] == 'partial_key':
                partial_key_r = int((str(data.decode('utf-8')).split(' '))[1])
                full_key = shr.generate_full_key(partial_key_r)    
            if not data:
                print(f'{datetime.datetime.now()} INFO: Соединение с {addr} разорвано')
                break
            data = shr.decrypt_message(data.decode('utf-8'))
            data = data.upper()
            data = shr.encrypt_message(data)
            print(f'{datetime.datetime.now()} INFO: Отправлены данные клиенту:', data)
            conn.send(bytes(data, 'utf-8'))
        print(f'{datetime.datetime.now()} INFO: Закрываем соединение с {addr}')
        conn.close()
    
    
    def listen(self, num = 10):
        self.socket.listen(num)
        while True:
            client_conn, client_addr = self.socket.accept()
            print('INFO: Создано подключение:', client_addr)
            threading.Thread(target = self.client_processing, args = (client_conn, client_addr))

In [4]:
public_key = 123
private_key = 189
server = ThreadedServer(public_key,private_key)

In [ ]:
server.listen(5)